# Relaxed Search(제대로) 가미한 코드

### 5월 30일 업데이트

<strong> 이 코드 예상 실행 시간... 적으면 15분... 많으면 20분?(데스크탑 기준...)<br>데이터 양에 따라 차이가 남.</strong>



<strong> 참고로 밑에 스스로 판단해서 parameter 넣어하는 칸이 하나 있음.<br>한번 쭉 밀고 그 셀 채우고 다시 쭉 미셈</strong>
<br>
<strong> 아 그리고 XGboost 는 계속 Warning 이 뜨는데 이건 뭔 짓을 해도 뭐 없앨 수가 없네...</strong>

In [1]:
# 최적의 패러미터 가진 모델을 몇번 돌릴지 값.
test_times=20
# random 인 max_state 를 설정하기 위한 변수.
full_count=0 # 건들지 말것.
# test size
ts = 0.2
# alpha value (밑에 정규검사 등에서 이용되는 p 값)
alpha = 0.05

In [2]:
#데이터 파일 업로드
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from numpy import mean
from numpy import std
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy import stats
import xgboost as xgb
from scipy.stats import wilcoxon
import scipy.stats
import random
random.seed(42)
import warnings
np.set_printoptions(precision=4,suppress=True)
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',500)
plt.rcParams["font.family"]='Malgun Gothic'

아래 셀에서 데이터 수정. <strong> 앞에 Data/ 있어야 제대로 됨</strong><br>
원하는 데이터 이름만 집어넣으면 뒤에는 알아서 될거임

In [3]:
dataname = "bundang3years_large1.csv"

In [4]:
#데이터 확인 및 필요 없는 열 삭제
pd.set_option('display.max_columns', 500)
df=pd.read_csv(dataname)
df.drop(['시군구'],axis=1,inplace=True)
df.drop(['단지명'],axis=1,inplace=True)
df.drop(['건축년도'],axis=1,inplace=True)

Y = df['거래금액'].values
Y = Y.astype('float')
X = df.drop(labels=['거래금액'],axis=1)
X = X.astype('float')

### 1. Random Forest

n_estimators = number of trees in the foreset<br>
max_features = max number of features considered for splitting a node<br>
max_depth = max number of levels in each decision tree<br>
min_samples_split = min number of data points placed in a node before the node is split<br>
min_samples_leaf = min number of data points allowed in a leaf node<br>
bootstrap = method for sampling data points (with or without replacement)

In [5]:
Tree_nums = [5,10,50,100,250,500,750]
Var_nums = [1,3,5,7]

# Random Forest Relaxed Search
x = len(Tree_nums)
y = len(Var_nums)
count=0

RF_RMSE_train_list =[]
RF_RMSE_test_list =[]
RF_MAPE_train_list =[]
RF_MAPE_test_list =[]
RF_par_diff_list=[]
RF_par_trees_list=[]
RF_par_var_list=[]
RF_min_vars_trees_list=[]
RF_min_vars_var_list=[]   

print("----progress...----")

for a in Tree_nums:
    for b in Var_nums:
        RF_temp_RMSE_train=0
        RF_temp_RMSE_test=0
        RF_temp_MAPE_train=0
        RF_temp_MAPE_test=0
        RF_temp_RMSE_diff=0
        for i in range(test_times):
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
        
            RF_model = RandomForestRegressor(n_estimators=a, max_depth=b,random_state=full_count)
            RF_model.fit(X_train, Y_train)
        
            RF_train_test = RF_model.predict(X_train)
            RF_train_test = RF_train_test.astype('int')
            
            RF_prediction_test = RF_model.predict(X_test)
            RF_prediction_test = RF_prediction_test.astype('int')   
            
            RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
            RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
            RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2)
            RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)
            RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)
            
            RF_temp_RMSE_train+=RF_RMSE_train
            RF_temp_RMSE_test+=RF_RMSE_test
            RF_temp_MAPE_train+=RF_MAPE_train
            RF_temp_MAPE_test+=RF_MAPE_test
            RF_temp_RMSE_diff+=RF_RMSE_diff
            full_count+=1
            count+=1
            progress = round(count/(x*y*test_times)*100,2)
            print(round(progress), end = ", ")
#             print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,ts,a,b,RF_RMSE_diff,RF_RMSE_train,RF_RMSE_test,RF_MAPE_train,RF_MAPE_test))
             
        RF_RMSE_train_list.append(RF_temp_RMSE_train/test_times)
        RF_RMSE_test_list.append(RF_temp_RMSE_test/test_times)
        RF_MAPE_train_list.append(RF_temp_MAPE_train/test_times)
        RF_MAPE_test_list.append(RF_temp_MAPE_test/test_times)
        RF_par_diff_list.append(RF_temp_RMSE_diff/test_times)
        RF_par_trees_list.append(a)
        RF_par_var_list.append(b)

for i in range(5):
    val = min(RF_par_diff_list)
    index = RF_par_diff_list.index(val)
    RF_par_diff_list.pop(index)
    RF_min_vars_trees_list.append(RF_par_trees_list.pop(index))
    RF_min_vars_var_list.append(RF_par_var_list.pop(index))
    
    
# RF_min_vars_trees_list
# RF_min_vars_var_list

----progress...----
0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 46, 46, 46, 46, 4

In [6]:
print("...testing...")

progress=0
count=0
RF_Best_RMSE_train_list=[]
RF_Best_RMSE_test_list=[]
RF_Best_MAPE_train_list=[]
RF_Best_MAPE_test_list=[]
RF_Best_RMSE_diff_list=[]

RF_Best_total_data=[]

for i in range(5):
    temp_RF_Best_RMSE_train_list=[]
    temp_RF_Best_RMSE_test_list=[]
    temp_RF_Best_MAPE_train_list=[]
    temp_RF_Best_MAPE_test_list=[]
    temp_RF_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = RF_min_vars_trees_list[i]
        b = RF_min_vars_var_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        RF_Best_model = RandomForestRegressor(n_estimators=a, max_depth=b,random_state=full_count )
        RF_Best_model.fit(X_train, Y_train)

        RF_Best_prediction_test = RF_Best_model.predict(X_test)
        RF_Best_prediction_test = RF_Best_prediction_test.astype('int')

        RF_Best_train_test = RF_Best_model.predict(X_train)
        RF_Best_train_test = RF_Best_train_test.astype('int')

        RF_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_Best_train_test)).astype('int')
        RF_Best_MAPE_train = round(np.mean(np.abs((Y_train - RF_Best_train_test) / Y_train)) * 100 ,2)

        RF_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_Best_prediction_test)).astype('int')
        RF_Best_MAPE_test = round(np.mean(np.abs((Y_test - RF_Best_prediction_test) / Y_test)) * 100 ,2)

        RF_Best_RMSE_diff = np.abs((RF_Best_RMSE_test-RF_Best_RMSE_train)/RF_Best_RMSE_test *100)
        
        temp_RF_Best_RMSE_train_list.append(RF_Best_RMSE_train)
        temp_RF_Best_RMSE_test_list.append(RF_Best_RMSE_test)
        temp_RF_Best_MAPE_train_list.append(RF_Best_MAPE_train)
        temp_RF_Best_MAPE_test_list.append(RF_Best_MAPE_test)
        temp_RF_Best_RMSE_diff_list.append(RF_Best_RMSE_diff)
        
        full_count+=1
        count+=1
        progress = round(count/(10*5)*100,2)
#         print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,RF_Best_RMSE_diff,RF_Best_RMSE_train,RF_Best_RMSE_test,RF_Best_MAPE_train,RF_Best_MAPE_test))
    
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(mean(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_train_list))
    RF_Best_total_data.append(std(temp_RF_Best_MAPE_test_list))
    RF_Best_total_data.append(mean(temp_RF_Best_RMSE_diff_list))
    RF_Best_total_data.append(std(temp_RF_Best_RMSE_diff_list))


...testing...


In [7]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("Tree_num : {}, Var_num : {}".format(RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))

rf1=[]
rf2=[]
rf3=[]
rf4=[]
rf5=[]

for i in range(50):
    k=i//10
    if k==0:
        rf1.append(RF_Best_total_data[i])
    if k==1:
        rf2.append(RF_Best_total_data[i])
    if k==2:
        rf3.append(RF_Best_total_data[i])
    if k==3:
        rf4.append(RF_Best_total_data[i])
    if k==4:
        rf5.append(RF_Best_total_data[i])
    
RF_s1 = pd.Series(rf1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

RF_s2 = pd.Series(rf2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
RF_s3 = pd.Series(rf3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
RF_s4 = pd.Series(rf4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
RF_s5 = pd.Series(rf5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

RF_best_total_dataframe = pd.concat([RF_s1,RF_s2,RF_s3,RF_s4,RF_s5],axis=1)
RF_best_total_dataframe

PARAMETERS FOR BEST RF...
Tree_num : 750, Var_num : 3
Tree_num : 500, Var_num : 1
Tree_num : 50, Var_num : 1
Tree_num : 250, Var_num : 1
Tree_num : 10, Var_num : 1


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,11220.150000,20337.600000,19915.400000,20122.900000,20657.550000
RMSE test Mean,14523.950000,19438.600000,20664.250000,20375.650000,19309.550000
RMSE train Stdev,562.858532,833.729836,1160.284293,988.655193,675.379928
RMSE test Stdev,2894.171237,2142.123348,3190.103695,2662.706861,1989.305318
MAPE train Mean,15.429000,31.339500,31.085000,31.366500,31.710500
MAPE test Mean,16.113500,31.668500,30.955500,31.173500,32.305500
MAPE train Stdev,0.338746,0.833651,1.318603,0.920925,1.441969
MAPE test Stdev,1.324977,1.511841,2.368243,2.060595,2.428344
RMSE diff(%) Mean,21.361027,11.864927,15.716055,13.686498,12.230946
RMSE diff(%) Stdev,15.650089,8.029997,9.529080,8.601628,7.851133


### 2. Support Vector Machine parameters

In [8]:
gamma_nums = [0.00005,0.0001,0.000125,0.00015,0.000175,0.0002,]
C_values= [5000,75000,10000,12500,15000]
x = len(gamma_nums)
y = len(C_values)
count=0

SVR_RMSE_train_list =[]
SVR_RMSE_test_list =[]
SVR_MAPE_train_list =[]
SVR_MAPE_test_list =[]
SVR_par_diff_list=[]
SVR_par_gamma_list=[]
SVR_par_cval_list=[]
SVR_min_vars_gamma_list=[]
SVR_min_vars_cval_list=[]   

print("----progress...----")

for a in gamma_nums:
    for b in C_values:
        SVR_temp_RMSE_train=0
        SVR_temp_RMSE_test=0
        SVR_temp_MAPE_train=0
        SVR_temp_MAPE_test=0
        SVR_temp_RMSE_diff=0
        for i in range(test_times): 
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

            SVR_model = svm.SVR(kernel='rbf',gamma=a,C=b)
            SVR_model.fit(X_train, Y_train)
            
            SVR_prediction_test = SVR_model.predict(X_test)
            SVR_prediction_test = SVR_prediction_test.astype('int')
            
            SVR_train_test = SVR_model.predict(X_train)
            SVR_train_test = SVR_train_test.astype('int')
            
            SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
            SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
            SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
            SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
            SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)

            SVR_temp_RMSE_train+=SVR_RMSE_train
            SVR_temp_RMSE_test+=SVR_RMSE_test
            SVR_temp_MAPE_train+=SVR_MAPE_train
            SVR_temp_MAPE_test+=SVR_MAPE_test
            SVR_temp_RMSE_diff+=SVR_RMSE_diff
            
            full_count+=1
            count+=1
            progress = round(count/(x*y*test_times)*100,2)
            print(round(progress), end = ", ")
#             print(" {:.2f}% , case:{} , Gamma_size: {} ,  C_values: {} , difference(%) : {:.2f}\n "
#             "---  train RMSE : {} , test RMSE : {}\n"
#             "---  train MAPE : {} , test MAPE : {}"
#                   .format(progress,count,a,b,SVR_RMSE_diff,SVR_RMSE_train,SVR_RMSE_test,SVR_MAPE_train,SVR_MAPE_test))            
            
        SVR_RMSE_train_list.append(SVR_RMSE_train)
        SVR_RMSE_test_list.append(SVR_RMSE_test)
        SVR_MAPE_train_list.append(SVR_MAPE_train)
        SVR_MAPE_test_list.append(SVR_MAPE_test)
        SVR_par_diff_list.append(SVR_RMSE_diff)
        SVR_par_gamma_list.append(a)
        SVR_par_cval_list.append(b) 

for i in range(5):
    val = min(SVR_par_diff_list)
    index = SVR_par_diff_list.index(val)
    SVR_par_diff_list.pop(index)
    SVR_min_vars_gamma_list.append(SVR_par_gamma_list.pop(index))
    SVR_min_vars_cval_list.append(SVR_par_cval_list.pop(index))

----progress...----
0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 

In [9]:
print("\n\n...testing...\n")
progress=0
count=0
SVR_Best_RMSE_train_list=[]
SVR_Best_RMSE_test_list=[]
SVR_Best_MAPE_train_list=[]
SVR_Best_MAPE_test_list=[]
SVR_Best_RMSE_diff_list=[]

SVR_Best_total_data=[]

for i in range(5):
    temp_SVR_Best_RMSE_train_list=[]
    temp_SVR_Best_RMSE_test_list=[]
    temp_SVR_Best_MAPE_train_list=[]
    temp_SVR_Best_MAPE_test_list=[]
    temp_SVR_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = SVR_min_vars_gamma_list[i]
        b = SVR_min_vars_cval_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts,random_state=full_count)

        SVR_Best_model = svm.SVR(kernel='rbf',gamma=a,C=b)
        SVR_Best_model.fit(X_train, Y_train)
            
        SVR_Best_prediction_test = SVR_Best_model.predict(X_test)
        SVR_Best_prediction_test = SVR_Best_prediction_test.astype('int')
            
        SVR_Best_train_test = SVR_Best_model.predict(X_train)
        SVR_Best_train_test = SVR_Best_train_test.astype('int')
            
        SVR_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_Best_train_test)).astype('int')
        SVR_Best_MAPE_train = round(np.mean(np.abs((Y_train - SVR_Best_train_test) / Y_train)) * 100 ,2)
            
        SVR_Best_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_Best_prediction_test)).astype('int')
        SVR_Best_MAPE_test = round(np.mean(np.abs((Y_test - SVR_Best_prediction_test) / Y_test)) * 100 ,2)
            
        SVR_Best_RMSE_diff = np.abs((SVR_Best_RMSE_test-SVR_Best_RMSE_train)/SVR_Best_RMSE_test *100)
        
        temp_SVR_Best_RMSE_train_list.append(SVR_Best_RMSE_train)
        temp_SVR_Best_RMSE_test_list.append(SVR_Best_RMSE_test)
        temp_SVR_Best_MAPE_train_list.append(SVR_Best_MAPE_train)
        temp_SVR_Best_MAPE_test_list.append(SVR_Best_MAPE_test)
        temp_SVR_Best_RMSE_diff_list.append(SVR_Best_RMSE_diff)
        full_count+=1  
        count+=1
#         progress = round(count/(test_times*5)*100,2)
#         print(" {:.2f}% , case:{} , Tree_num: {} , Var_num: {} , difference(%) : {:.2f}\n "
#                 "---  train RMSE : {} , test RMSE : {}\n"
#                 "---  train MAPE : {} , test MAPE : {}"
#                       .format(progress,count,ts,a,b,SVR_Best_RMSE_diff,SVR_Best_RMSE_train,SVR_Best_RMSE_test,SVR_Best_MAPE_train,SVR_Best_MAPE_test))
    
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_train_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_MAPE_test_list))
    SVR_Best_total_data.append(mean(temp_SVR_Best_RMSE_diff_list))
    SVR_Best_total_data.append(std(temp_SVR_Best_RMSE_diff_list))

    
    




...testing...



In [10]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))

SVR1=[]
SVR2=[]
SVR3=[]
SVR4=[]
SVR5=[]

for i in range(50):
    k=i//10
    if k==0:
        SVR1.append(SVR_Best_total_data[i])
    if k==1:
        SVR2.append(SVR_Best_total_data[i])
    if k==2:
        SVR3.append(SVR_Best_total_data[i])
    if k==3:
        SVR4.append(SVR_Best_total_data[i])
    if k==4:
        SVR5.append(SVR_Best_total_data[i])

    
SVR_s1 = pd.Series(SVR1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

SVR_s2 = pd.Series(SVR2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
SVR_s3 = pd.Series(SVR3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
SVR_s4 = pd.Series(SVR4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
SVR_s5 = pd.Series(SVR5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')

SVR_best_total_dataframe_rbf = pd.concat([SVR_s1,SVR_s2,SVR_s3,SVR_s4,SVR_s5],axis=1)
SVR_best_total_dataframe_rbf

PARAMETERS FOR BEST SVR...
Gamma_size: 0.00015 ,  C_values: 5000
Gamma_size: 0.00015 ,  C_values: 10000
Gamma_size: 0.0002 ,  C_values: 10000
Gamma_size: 5e-05 ,  C_values: 5000
Gamma_size: 0.0001 ,  C_values: 12500


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,12052.700000,10897.050000,11023.050000,12743.700000,10392.150000
RMSE test Mean,11791.150000,10868.600000,10364.400000,11441.600000,11969.600000
RMSE train Stdev,1270.405569,1518.624525,1424.848219,1018.052410,1380.351161
RMSE test Stdev,3776.303792,4510.688766,4441.618775,3501.616390,4460.111528
MAPE train Mean,7.228000,6.141000,5.965000,9.416000,6.211000
MAPE test Mean,8.397000,7.304000,7.088500,10.047500,7.255000
MAPE train Stdev,0.182060,0.145461,0.135259,0.150579,0.134309
MAPE test Stdev,0.723306,0.536763,0.607892,0.744640,0.599162
RMSE diff(%) Mean,29.886743,43.112488,48.382060,38.085108,40.220315
RMSE diff(%) Stdev,22.376696,22.197960,21.606335,13.203521,18.743873


### 3. XGBoost ... <br>
DAMN...

In [11]:
feattree=[0.2,0.4,0.6]
learnrate=[0.1,0.2,0.3,0.4]
maxdepth=[5,7,9,11]
trees=[50,75,100,250,500]
count=0

XGB_RMSE_train_list =[]
XGB_RMSE_test_list =[]
XGB_MAPE_train_list =[]
XGB_MAPE_test_list =[]
XGB_par_diff_list=[]

XGB_par_feattree_list=[]
XGB_par_learnrate_list=[]
XGB_par_maxdepth_list=[]
XGB_par_trees_list=[]

XGB_min_vars_feattree_list=[]
XGB_min_vars_learnrate_list=[]  
XGB_min_vars_maxdepth_list=[]
XGB_min_vars_trees_list=[] 

print("----progress...----")

for a in feattree:
    for b in learnrate:
        for c in maxdepth:
            for d in trees:
                X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
                xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = a, 
                                              learning_rate = b, max_depth = c, n_estimators = d)
                xg_reg.fit(X_train,Y_train)
                    
                XGB_train_test = xg_reg.predict(X_train).astype('int')
                XGB_prediction_test = xg_reg.predict(X_test).astype('int')
                    
                XGB_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_train_test)).astype('int')
                XGB_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_prediction_test)).astype('int')
                XGB_MAPE_train = round(np.mean(np.abs((Y_train - XGB_train_test) / Y_train)) * 100 ,2)
                XGB_MAPE_test = round(np.mean(np.abs((Y_test - XGB_prediction_test) / Y_test)) * 100 ,2)
                XGB_RMSE_diff = np.abs((XGB_RMSE_test-XGB_RMSE_train)/XGB_RMSE_test *100)

                full_count+=1
                count+=1
                    
#                 progress = round(count/(len(feattree)*len(learnrate)*len(maxdepth)*len(trees))*100,2)
#                 print(round(progress), end = ", ")
                
#                 print("{} {} {} {} \n{} {} {} {} {}".
#                     format(a, b, c,d,
#                         XGB_RMSE_train,XGB_RMSE_test,XGB_MAPE_train,XGB_MAPE_test,round(XGB_RMSE_diff,2) ))
                
                
                XGB_RMSE_train_list.append(XGB_RMSE_train)
                XGB_RMSE_test_list.append(XGB_RMSE_test)
                XGB_MAPE_train_list.append(XGB_MAPE_train)
                XGB_MAPE_test_list.append(XGB_MAPE_test)
                XGB_par_diff_list.append(XGB_RMSE_diff)
                
                XGB_par_feattree_list.append(a)
                XGB_par_learnrate_list.append(b)
                XGB_par_maxdepth_list.append(c)
                XGB_par_trees_list.append(d)

sp_count=0
while sp_count < 5:
    val = min(XGB_par_diff_list)
    index = XGB_par_diff_list.index(val)
    XGB_par_diff_list.pop(index)
    if((XGB_MAPE_train_list[index])<=15 and (XGB_MAPE_test_list[index])<=15):
        XGB_min_vars_feattree_list.append(XGB_par_feattree_list.pop(index))
        XGB_min_vars_learnrate_list.append(XGB_par_learnrate_list.pop(index))
        XGB_min_vars_maxdepth_list.append(XGB_par_maxdepth_list.pop(index))
        XGB_min_vars_trees_list.append(XGB_par_trees_list.pop(index))
        sp_count+=1

----progress...----
[15:26:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:26:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:26:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:26:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:26:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:26:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/

In [12]:
print("PARAMETERS FOR BEST XGB...")
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

print("...testing...")

progress=0
count=0
XGB_Best_RMSE_train_list=[]
XGB_Best_RMSE_test_list=[]
XGB_Best_MAPE_train_list=[]
XGB_Best_MAPE_test_list=[]
XGB_Best_RMSE_diff_list=[]

XGB_Best_total_data=[]

for i in range(5):
    temp_XGB_Best_RMSE_train_list=[]
    temp_XGB_Best_RMSE_test_list=[]
    temp_XGB_Best_MAPE_train_list=[]
    temp_XGB_Best_MAPE_test_list=[]
    temp_XGB_Best_RMSE_diff_list=[]
    for k in range(test_times):
        a = XGB_min_vars_feattree_list[i]
        b = XGB_min_vars_learnrate_list[i]
        c = XGB_min_vars_maxdepth_list[i]
        d = XGB_min_vars_trees_list[i]

        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
        xg_Best_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = a, 
                                learning_rate = b, max_depth = c, n_estimators = d)
        
        xg_Best_reg.fit(X_train,Y_train)
                    
        XGB_Best_train_test = xg_Best_reg.predict(X_train).astype('int')
        XGB_Best_prediction_test = xg_Best_reg.predict(X_test).astype('int')
                    
        XGB_Best_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_Best_train_test)).astype('int')
        XGB_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_Best_prediction_test)).astype('int')

        XGB_Best_RMSE_train = np.sqrt(mean_squared_error(Y_train, XGB_Best_train_test)).astype('int')
        XGB_Best_MAPE_train = round(np.mean(np.abs((Y_train - XGB_Best_train_test) / Y_train)) * 100 ,2)

        XGB_Best_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_Best_prediction_test)).astype('int')
        XGB_Best_MAPE_test = round(np.mean(np.abs((Y_test - XGB_Best_prediction_test) / Y_test)) * 100 ,2)

        XGB_Best_RMSE_diff = np.abs((XGB_Best_RMSE_test-XGB_Best_RMSE_train)/XGB_Best_RMSE_test *100)
        
        temp_XGB_Best_RMSE_train_list.append(XGB_Best_RMSE_train)
        temp_XGB_Best_RMSE_test_list.append(XGB_Best_RMSE_test)
        temp_XGB_Best_MAPE_train_list.append(XGB_Best_MAPE_train)
        temp_XGB_Best_MAPE_test_list.append(XGB_Best_MAPE_test)
        temp_XGB_Best_RMSE_diff_list.append(XGB_Best_RMSE_diff)
        
        full_count+=1
        count+=1
        progress = round(count/(5*test_times)*100,2)

    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_train_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_test_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_train_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_test_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_MAPE_train_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_MAPE_test_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_MAPE_train_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_MAPE_test_list))
    XGB_Best_total_data.append(mean(temp_XGB_Best_RMSE_diff_list))
    XGB_Best_total_data.append(std(temp_XGB_Best_RMSE_diff_list))


PARAMETERS FOR BEST XGB...
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 75 
 feature % : 0.2, learn_rate : 0.2, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 9, trees : 50 
...testing...
[15:28:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:28:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:28:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[15:28:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:

In [13]:
print("PARAMETERS FOR BEST XGB...")
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

XGB1=[]
XGB2=[]
XGB3=[]
XGB4=[]
XGB5=[]

for i in range(50):
    k=i//10
    if k==0:
        XGB1.append(XGB_Best_total_data[i])
    if k==1:
        XGB2.append(XGB_Best_total_data[i])
    if k==2:
        XGB3.append(XGB_Best_total_data[i])
    if k==3:
        XGB4.append(XGB_Best_total_data[i])
    if k==4:
        XGB5.append(XGB_Best_total_data[i])
    
XGB_s1 = pd.Series(XGB1,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 1')

XGB_s2 = pd.Series(XGB2,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 2')
XGB_s3 = pd.Series(XGB3,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 3')
XGB_s4 = pd.Series(XGB4,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 4')
XGB_s5 = pd.Series(XGB5,
index=['RMSE train Mean','RMSE test Mean','RMSE train Stdev','RMSE test Stdev',
       'MAPE train Mean','MAPE test Mean','MAPE train Stdev','MAPE test Stdev',
       'RMSE diff(%) Mean','RMSE diff(%) Stdev'],
name='Model 5')


XGB_best_total_dataframe = pd.concat([XGB_s1,XGB_s2,XGB_s3,XGB_s4,XGB_s5],axis=1)
print("PARAMETERS FOR BEST XGB...")
XGB_best_total_dataframe

PARAMETERS FOR BEST XGB...
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 75 
 feature % : 0.2, learn_rate : 0.2, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 9, trees : 50 
PARAMETERS FOR BEST XGB...


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,7476.850000,6057.150000,5155.900000,5442.700000,6901.950000
RMSE test Mean,12437.900000,10644.050000,11209.700000,10274.200000,13330.550000
RMSE train Stdev,427.536346,204.858311,232.501591,262.222634,391.502807
RMSE test Stdev,3666.272411,3826.037826,3805.529189,3351.918803,4006.739204
MAPE train Mean,7.218500,6.400000,5.290500,5.995500,6.554000
MAPE test Mean,9.772000,8.280500,8.282000,8.487000,9.770500
MAPE train Stdev,0.196527,0.142513,0.140160,0.196710,0.136616
MAPE test Stdev,0.591537,0.556745,0.707217,0.589467,0.631961
RMSE diff(%) Mean,34.558154,35.187921,47.901478,42.044052,41.988280
RMSE diff(%) Stdev,19.519626,22.772743,18.575309,16.704498,21.249937


# COMPARING ALL FOUR IN ONE

In [14]:
print("PARAMETERS FOR BEST RF...")
for i in range(5):
    print("Tree_num : {}, Var_num : {}".format(RF_min_vars_trees_list[i],RF_min_vars_var_list[i]))
RF_best_total_dataframe

PARAMETERS FOR BEST RF...
Tree_num : 750, Var_num : 3
Tree_num : 500, Var_num : 1
Tree_num : 50, Var_num : 1
Tree_num : 250, Var_num : 1
Tree_num : 10, Var_num : 1


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,11220.150000,20337.600000,19915.400000,20122.900000,20657.550000
RMSE test Mean,14523.950000,19438.600000,20664.250000,20375.650000,19309.550000
RMSE train Stdev,562.858532,833.729836,1160.284293,988.655193,675.379928
RMSE test Stdev,2894.171237,2142.123348,3190.103695,2662.706861,1989.305318
MAPE train Mean,15.429000,31.339500,31.085000,31.366500,31.710500
MAPE test Mean,16.113500,31.668500,30.955500,31.173500,32.305500
MAPE train Stdev,0.338746,0.833651,1.318603,0.920925,1.441969
MAPE test Stdev,1.324977,1.511841,2.368243,2.060595,2.428344
RMSE diff(%) Mean,21.361027,11.864927,15.716055,13.686498,12.230946
RMSE diff(%) Stdev,15.650089,8.029997,9.529080,8.601628,7.851133


In [15]:
print("PARAMETERS FOR BEST SVR...")
for i in range(5):
    print("Gamma_size: {} ,  C_values: {}".format(SVR_min_vars_gamma_list[i],SVR_min_vars_cval_list[i]))
SVR_best_total_dataframe_rbf

PARAMETERS FOR BEST SVR...
Gamma_size: 0.00015 ,  C_values: 5000
Gamma_size: 0.00015 ,  C_values: 10000
Gamma_size: 0.0002 ,  C_values: 10000
Gamma_size: 5e-05 ,  C_values: 5000
Gamma_size: 0.0001 ,  C_values: 12500


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,12052.700000,10897.050000,11023.050000,12743.700000,10392.150000
RMSE test Mean,11791.150000,10868.600000,10364.400000,11441.600000,11969.600000
RMSE train Stdev,1270.405569,1518.624525,1424.848219,1018.052410,1380.351161
RMSE test Stdev,3776.303792,4510.688766,4441.618775,3501.616390,4460.111528
MAPE train Mean,7.228000,6.141000,5.965000,9.416000,6.211000
MAPE test Mean,8.397000,7.304000,7.088500,10.047500,7.255000
MAPE train Stdev,0.182060,0.145461,0.135259,0.150579,0.134309
MAPE test Stdev,0.723306,0.536763,0.607892,0.744640,0.599162
RMSE diff(%) Mean,29.886743,43.112488,48.382060,38.085108,40.220315
RMSE diff(%) Stdev,22.376696,22.197960,21.606335,13.203521,18.743873


In [16]:
for i in range(5):
    print(" feature % : {}, learn_rate : {}, max_depth : {}, trees : {} ".
          format(XGB_min_vars_feattree_list[i],XGB_min_vars_learnrate_list[i],XGB_min_vars_maxdepth_list[i],XGB_min_vars_trees_list[i]))

XGB_best_total_dataframe

 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 5, trees : 75 
 feature % : 0.2, learn_rate : 0.1, max_depth : 7, trees : 75 
 feature % : 0.2, learn_rate : 0.2, max_depth : 5, trees : 50 
 feature % : 0.2, learn_rate : 0.1, max_depth : 9, trees : 50 


,Model 1,Model 2,Model 3,Model 4,Model 5
RMSE train Mean,7476.850000,6057.150000,5155.900000,5442.700000,6901.950000
RMSE test Mean,12437.900000,10644.050000,11209.700000,10274.200000,13330.550000
RMSE train Stdev,427.536346,204.858311,232.501591,262.222634,391.502807
RMSE test Stdev,3666.272411,3826.037826,3805.529189,3351.918803,4006.739204
MAPE train Mean,7.218500,6.400000,5.290500,5.995500,6.554000
MAPE test Mean,9.772000,8.280500,8.282000,8.487000,9.770500
MAPE train Stdev,0.196527,0.142513,0.140160,0.196710,0.136616
MAPE test Stdev,0.591537,0.556745,0.707217,0.589467,0.631961
RMSE diff(%) Mean,34.558154,35.187921,47.901478,42.044052,41.988280
RMSE diff(%) Stdev,19.519626,22.772743,18.575309,16.704498,21.249937


아래 셀은 값들을 직접 입력할것.<br>
위에 표 보고 제일 괜찮을 것 같은거 스스로 판단해서<br>
그 parameter 들을 아래에다 넣자.<br>
그리고 다시 쭉 밀면 된다.<br>

In [5]:
RF_trees= 750
RF_depth= 3
SVR_gamma= 0.0002
SVR_c= 10000
XGB_colsample=0.2
XGB_learn_rate=0.1
XGB_max_depth=5
XGB_trees=75

In [6]:
final_RF_RMSE_train=[]
final_RF_RMSE_test=[]
final_RF_MAPE_train=[]
final_RF_MAPE_test=[]
final_RF_RMSE_diff=[]

final_SVR_RMSE_train=[]
final_SVR_RMSE_test=[]
final_SVR_MAPE_train=[]
final_SVR_MAPE_test=[]
final_SVR_RMSE_diff=[]

final_XGB_RMSE_train=[]
final_XGB_RMSE_test=[]
final_XGB_MAPE_train=[]
final_XGB_MAPE_test=[]
final_XGB_RMSE_diff=[]

final_LR_RMSE_train=[]
final_LR_RMSE_test=[]
final_LR_MAPE_train=[]
final_LR_MAPE_test=[]
final_LR_RMSE_diff=[]

count=0
for i in range(test_times):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=ts, random_state=full_count)
    
    # RF
    
    RF_model = RandomForestRegressor(n_estimators=RF_trees, max_depth=RF_depth,random_state=full_count)
    RF_model.fit(X_train, Y_train)      
    RF_prediction_test = RF_model.predict(X_test).astype('int')  
    RF_train_test = RF_model.predict(X_train).astype('int') 
    
    RF_RMSE_train =np.sqrt(mean_squared_error(Y_train, RF_train_test)).astype('int')
    RF_RMSE_test = np.sqrt(mean_squared_error(Y_test, RF_prediction_test)).astype('int')
    RF_MAPE_train = round(np.mean(np.abs((Y_train - RF_train_test) / Y_train)) * 100 ,2) 
    RF_MAPE_test = round(np.mean(np.abs((Y_test - RF_prediction_test) / Y_test)) * 100 ,2)        
    RF_RMSE_diff = np.abs((RF_RMSE_test-RF_RMSE_train)/RF_RMSE_test *100)      
    
    final_RF_RMSE_train.append(RF_RMSE_train)
    final_RF_RMSE_test.append(RF_RMSE_test)
    final_RF_MAPE_train.append(RF_MAPE_train)
    final_RF_MAPE_test.append(RF_MAPE_test)
    final_RF_RMSE_diff.append(RF_RMSE_diff)
    
    full_count+=1
    
    # SVR
    
    SVR_model = svm.SVR(kernel='rbf',gamma=SVR_gamma,C=SVR_c)
    SVR_model.fit(X_train, Y_train)
    SVR_prediction_test = SVR_model.predict(X_test).astype('int')       
    SVR_train_test = SVR_model.predict(X_train).astype('int')
    
    SVR_RMSE_train =np.sqrt(mean_squared_error(Y_train, SVR_train_test)).astype('int')
    SVR_RMSE_test =np.sqrt(mean_squared_error(Y_test, SVR_prediction_test)).astype('int')
    SVR_MAPE_train = round(np.mean(np.abs((Y_train - SVR_train_test) / Y_train)) * 100 ,2)
    SVR_MAPE_test = round(np.mean(np.abs((Y_test - SVR_prediction_test) / Y_test)) * 100 ,2)
    SVR_RMSE_diff = np.abs((SVR_RMSE_test-SVR_RMSE_train)/SVR_RMSE_test *100)
            
    final_SVR_RMSE_train.append(SVR_RMSE_train)
    final_SVR_RMSE_test.append(SVR_RMSE_test)
    final_SVR_MAPE_train.append(SVR_MAPE_train)
    final_SVR_MAPE_test.append(SVR_MAPE_test)        
    final_SVR_RMSE_diff.append(SVR_RMSE_diff)
    
    full_count+=1
        
    # XGboost 
        
    xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = XGB_colsample, learning_rate = XGB_learn_rate, 
                              max_depth = XGB_max_depth, n_estimators = XGB_trees)
    xg_reg.fit(X_train,Y_train)
    
    XGB_train_test = xg_reg.predict(X_train).astype('int')
    XGB_prediction_test = xg_reg.predict(X_test).astype('int')
                    
    XGB_RMSE_train =np.sqrt(mean_squared_error(Y_train, XGB_train_test)).astype('int')
    XGB_RMSE_test = np.sqrt(mean_squared_error(Y_test, XGB_prediction_test)).astype('int')
    XGB_MAPE_train = round(np.mean(np.abs((Y_train - XGB_train_test) / Y_train)) * 100 ,2)
    XGB_MAPE_test = round(np.mean(np.abs((Y_test - XGB_prediction_test) / Y_test)) * 100 ,2)
    XGB_RMSE_diff = np.abs((XGB_RMSE_test-XGB_RMSE_train)/XGB_RMSE_test *100)
    
    final_XGB_RMSE_train.append(XGB_RMSE_train)
    final_XGB_RMSE_test.append(XGB_RMSE_test)
    final_XGB_MAPE_train.append(XGB_MAPE_train)
    final_XGB_MAPE_test.append(XGB_MAPE_test)
    final_XGB_RMSE_diff.append(XGB_RMSE_diff)    

    full_count+=1

    # LR
    
    LR_model=LinearRegression()
    LR_model.fit(X_train, Y_train)
    LR_prediction_test = LR_model.predict(X_test).astype('int')
    LR_train_test = LR_model.predict(X_train).astype('int')

    
    LR_RMSE_train =np.sqrt(mean_squared_error(Y_train, LR_train_test)).astype('int')
    LR_RMSE_test =np.sqrt(mean_squared_error(Y_test, LR_prediction_test)).astype('int')    
    LR_MAPE_train = round(np.mean(np.abs((Y_train - LR_train_test) / Y_train)) * 100 ,2)
    LR_MAPE_test = round(np.mean(np.abs((Y_test - LR_prediction_test) / Y_test)) * 100 ,2)            
    LR_RMSE_diff = np.abs((LR_RMSE_test-LR_RMSE_train)/LR_RMSE_test *100)
    
    final_LR_RMSE_train.append(LR_RMSE_train)
    final_LR_RMSE_test.append(LR_RMSE_test)
    final_LR_MAPE_train.append(LR_MAPE_train)
    final_LR_MAPE_test.append(LR_MAPE_test)    
    final_LR_RMSE_diff.append(LR_RMSE_diff)
    
    full_count+=1


[20:46:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:46:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:46:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:46:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:46:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[20:46:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:17

In [7]:
print("RMSE values")
RMSE_test_all = pd.concat([pd.Series(final_RF_RMSE_test),pd.Series(final_SVR_RMSE_test),
                           pd.Series(final_XGB_RMSE_test),pd.Series(final_LR_RMSE_test),pd.Series(final_LR_RMSE_train)],axis=1)

RMSE_test_all = RMSE_test_all.rename({'0':'RF'},axis='columns')
RMSE_test_all.columns=['RF','SVR','XGB','LR','train']
RMSE_test_all

RMSE values


,RF,SVR,XGB,LR,train
0,11889,10438,10180,11045,11916
1,11444,8499,7126,9729,12193
2,13633,9758,11517,11837,11744
3,12561,10530,9466,11406,11824
4,20036,19702,17645,17555,9842
5,18961,18811,16804,16882,10171
6,17986,18123,16439,16087,10394
7,12107,10203,9432,10531,12027
8,12895,10928,10202,11536,11802
9,10597,6773,5473,9654,12232


In [21]:
RF_RMSE_mean = mean(RMSE_test_all['RF'])
SVR_RMSE_mean = mean(RMSE_test_all['SVR'])
XGB_RMSE_mean = mean(RMSE_test_all['XGB'])
LR_RMSE_mean = mean(RMSE_test_all['LR'])
print("RF_RMSE_mean : {} \nSVR_RMSE_mean : {} \nXGB_RMSE_mean : {} \nLR_RMSE_mean : {}".
      format(RF_RMSE_mean,SVR_RMSE_mean,XGB_RMSE_mean,LR_RMSE_mean))

print("max: {}".format(max(RF_RMSE_mean,SVR_RMSE_mean,XGB_RMSE_mean,LR_RMSE_mean)))


RF_RMSE_mean : 13018.15 
SVR_RMSE_mean : 9907.6 
XGB_RMSE_mean : 9603.35 
LR_RMSE_mean : 11117.3
max: 13018.15


In [8]:
print("MAPE values")
MAPE_test_all = pd.concat([pd.Series(final_RF_MAPE_test),pd.Series(final_SVR_MAPE_test),
                           pd.Series(final_XGB_MAPE_test),pd.Series(final_LR_MAPE_test),pd.Series(final_LR_MAPE_train)],axis=1)

MAPE_test_all = MAPE_test_all.rename({'0':'RF'},axis='columns')
MAPE_test_all.columns=['RF','SVR','XGB','LR','train']
MAPE_test_all

MAPE values


,RF,SVR,XGB,LR,train
0,13.69,7.17,8.83,13.20,13.46
1,16.06,7.91,8.20,13.18,13.40
2,17.87,8.22,9.77,14.86,12.91
3,15.47,7.08,8.39,13.73,13.29
4,16.01,7.62,8.85,12.84,12.90
5,15.01,7.63,8.55,13.33,13.14
6,14.72,6.67,8.57,12.99,13.31
7,15.61,7.89,8.74,13.96,13.35
8,16.14,7.69,9.17,14.20,13.06
9,15.82,6.09,6.72,13.50,13.60


In [23]:
RF_MAPE_mean = mean(MAPE_test_all['RF'])
SVR_MAPE_mean = mean(MAPE_test_all['SVR'])
XGB_MAPE_mean = mean(MAPE_test_all['XGB'])
LR_MAPE_mean = mean(MAPE_test_all['LR'])
print("RF_MAPE_mean : {} \nSVR_MAPE_mean : {} \nXGB_MAPE_mean : {} \nLR_MAPE_mean : {}".
      format(RF_MAPE_mean,SVR_MAPE_mean,XGB_MAPE_mean,LR_MAPE_mean))

print("max: {}".format(max(RF_MAPE_mean,SVR_MAPE_mean,XGB_MAPE_mean,LR_MAPE_mean)))

RF_MAPE_mean : 15.952999999999998 
SVR_MAPE_mean : 7.199 
XGB_MAPE_mean : 8.4245 
LR_MAPE_mean : 13.727
max: 15.952999999999998


In [24]:
print("diff values")
diff_test_all = pd.concat([pd.Series(final_RF_RMSE_diff),pd.Series(final_SVR_RMSE_diff),
                           pd.Series(final_XGB_RMSE_diff),pd.Series(final_LR_RMSE_diff)],axis=1)

diff_test_all = diff_test_all.rename({'0':'RF'},axis='columns')
diff_test_all.columns=['RF','SVR','XGB','LR']
diff_test_all


diff values


,RF,SVR,XGB,LR
0,16.902463,5.660728,34.992360,2.616481
1,18.095588,3.373404,38.171768,2.209897
2,1.154774,50.979381,14.473139,23.913043
3,16.982223,7.061611,36.201349,0.161895
4,14.703231,3.323263,37.740733,6.624238
5,16.983083,15.681592,24.520124,5.828358
6,26.652147,18.857314,50.549828,11.885688
7,6.484827,75.733019,15.781424,29.887736
8,18.915249,3.647556,32.755595,0.247187
9,25.130056,90.658475,13.381523,34.912991


In [25]:
RF_diff_mean = mean(diff_test_all['RF'])
SVR_diff_mean = mean(diff_test_all['SVR'])
XGB_diff_mean = mean(diff_test_all['XGB'])
LR_diff_mean = mean(diff_test_all['LR'])
print("RF_diff_mean : {} \nSVR_diff_mean : {} \nXGB_diff_mean : {} \nLR_diff_mean : {}".
      format(RF_diff_mean,SVR_diff_mean,XGB_diff_mean,LR_diff_mean))

print("max: {}".format(max(RF_diff_mean,SVR_diff_mean,XGB_diff_mean,LR_diff_mean)))


RF_diff_mean : 15.028495339319468 
SVR_diff_mean : 31.23468552067849 
XGB_diff_mean : 34.35220384996774 
LR_diff_mean : 14.286440582698669
max: 34.35220384996774


In [26]:
# 정규 검정. 오직 하나의 줄, array 를 이용할 것
def Ntest(df):
    stat,p = stats.normaltest(df)
    if p>alpha:
        return False
    else:
        return True
    
# 그냥 4개를 정규검정해서 list 로 True False 반환
def fourNtest(k):
    a = k['RF']
    b = k['SVR']
    c = k['XGB']
    d = k['LR']
    tlist=[]
    tlist.append(Ntest(a))
    tlist.append(Ntest(b))
    tlist.append(Ntest(c))
    tlist.append(Ntest(d))
    return tlist

# T-test. Array 로 시킬 것.
def Ttest(df1,df2):
    s,p=stats.ttest_rel(df1, df2)
    if p>alpha:
        return False
    else:
        return True

# Wilcoxon test. Array 로 정큐분포를 이루지 않을때의 paired t-test
def Wiltest(df1,df2):
    z, p = wilcoxon(df2 - df1)
    if p>alpha:
        return False
    else:
        return True

In [27]:
print(fourNtest(RMSE_test_all))

[True, True, True, True]


In [28]:
print(fourNtest(MAPE_test_all))

[False, False, False, False]


In [29]:
print(fourNtest(diff_test_all))

[False, False, False, False]


그냥 다 정규분포 안 따른다고 생각해도 될듯

In [30]:
# RMSE_test_all['RF']    RMSE_test_all['SVR']    RMSE_test_all['XGB']    RMSE_test_all['LR']
# MAPE_test_all['RF']    MAPE_test_all['SVR']    MAPE_test_all['XGB']    MAPE_test_all['LR']
# diff_test_all['RF']    diff_test_all['SVR']    diff_test_all['XGB']    diff_test_all['LR']



# RMSE 간 차이가 있는지를 확인
print("-----RMSE-----")
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['SVR']))
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['XGB']))
print(Wiltest(RMSE_test_all['RF'],RMSE_test_all['LR']))
print(Wiltest(RMSE_test_all['SVR'],RMSE_test_all['XGB']))
print(Wiltest(RMSE_test_all['SVR'],RMSE_test_all['LR']))
print(Wiltest(RMSE_test_all['XGB'],RMSE_test_all['LR']))

# MAPE 간 차이가 있는지를 확인
print("-----MAPE-----")
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['SVR']))
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['XGB']))
print(Wiltest(MAPE_test_all['RF'],MAPE_test_all['LR']))
print(Wiltest(MAPE_test_all['SVR'],MAPE_test_all['XGB']))
print(Wiltest(MAPE_test_all['SVR'],MAPE_test_all['LR']))
print(Wiltest(MAPE_test_all['XGB'],MAPE_test_all['LR']))

# diff 간 차이가 있는지를 확인
print("-----diff-----")
print(Wiltest(diff_test_all['RF'],diff_test_all['SVR']))
print(Wiltest(diff_test_all['RF'],diff_test_all['XGB']))
print(Wiltest(diff_test_all['RF'],diff_test_all['LR']))
print(Wiltest(diff_test_all['SVR'],diff_test_all['XGB']))
print(Wiltest(diff_test_all['SVR'],diff_test_all['LR']))
print(Wiltest(diff_test_all['XGB'],diff_test_all['LR']))

-----RMSE-----
True
True
True
False
True
True
-----MAPE-----
True
True
True
True
True
True
-----diff-----
False
True
False
False
True
True


결과: RMSE 든 MAPE 든 모델 간 성능 차이가 확실하다.